In [6]:
# %pip install seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [7]:
ds = pd.read_json("easy_dataset_1.json")
ds

,dataset_id,num_papers,num_reviewers,reviewer_capacity,min_reviews_per_paper,max_reviews_per_paper,preferences,friendships,authorship
0,Easy Dataset 1,5,5,3,3,5,"[3, 2, 1, 5, 3]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,Easy Dataset 1,5,5,3,3,5,"[5, 2, 4, 2, 2]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]"
2,Easy Dataset 1,5,5,3,3,5,"[4, 3, 1, 2, 4]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,Easy Dataset 1,5,5,3,3,5,"[4, 2, 2, 1, 1]","[0, 0, 0, 0, 0]","[1, 1, 0, 0, 1]"
4,Easy Dataset 1,5,5,3,3,5,"[4, 3, 4, 1, 5]","[0, 0, 0, 0, 0]","[0, 0, 1, 1, 0]"
